In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding,SimpleRNN, GRU,Activation,Flatten
from tensorflow.keras.layers import Conv1D,MaxPool1D, Flatten
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence
from sklearn.metrics import (precision_score, recall_score,confusion_matrix,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
import pandas as pd
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt

In [2]:
import tensorflow.keras
print('keras: %s' % tensorflow.keras.__version__)

keras: 2.11.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/unknown_labelled_data.csv')

In [5]:
df.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label_BENIGN,Label_DDoS,Label_DoS Hulk,Label_PortScan,Label_Unknown
0,0,-0.646799,0.324411,-1.948333,0.204423,2.428566,-0.662668,2.051947,-0.472445,-0.008076,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0
1,1,1.783219,0.430907,0.508829,0.114303,-0.421518,1.505849,-0.394452,-0.469837,-0.009562,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0
2,2,-0.302869,-0.414284,0.159378,-0.558287,-0.421518,1.505849,-0.416902,-0.472439,-0.009562,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0
3,3,-0.126277,-1.034865,0.769223,-1.040696,-0.421518,1.505849,-0.407956,-0.470938,-0.006590,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0
4,4,-0.708056,0.979845,-1.948333,-0.560275,2.462290,-0.662668,-0.401372,-0.472445,-0.011049,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0


In [6]:
df = df.sample(frac = 1)

In [7]:
df.drop(df.columns[0] , inplace=True , axis=1)

In [8]:
df.columns

Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  

In [9]:
LabelColumns = ['Label_BENIGN', 'Label_DDoS','Label_DoS Hulk', 'Label_PortScan', 'Label_Unknown']

In [10]:
y = df[LabelColumns]

In [11]:
y

,Label_BENIGN,Label_DDoS,Label_DoS Hulk,Label_PortScan,Label_Unknown
1216951,1,0,0,0,0
1359019,0,1,0,0,0
1232648,1,0,0,0,0
750384,1,0,0,0,0
1512173,0,0,1,0,0
...,...,...,...,...,...
599225,1,0,0,0,0
1040938,1,0,0,0,0
1275183,1,0,0,0,0
767898,1,0,0,0,0


In [12]:
X = df.drop(LabelColumns , axis=1)

In [13]:
X

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
1216951,-1.148078,-1.572186,0.801355,-1.376659,-0.399227,-0.662668,-0.419940,2.889282,0.090015,0.066965,...,0.007494,0.002729,0.182051,1.485880,2.017885,-0.066570,-0.040514,0.126217,-0.043919,-0.176794
1359019,-0.976250,-1.296264,-0.690631,-1.039370,-0.419975,-0.662668,-0.415552,-0.356160,-0.005104,-0.010221,...,-0.001222,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
1232648,-0.618347,0.483187,0.870511,0.202435,-0.421518,1.505849,0.998306,-0.471576,-0.011049,-0.009102,...,-0.008194,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
750384,1.816940,-0.589518,0.838709,-0.681540,-0.421518,1.505849,-0.414201,-0.471015,-0.006590,-0.005746,...,-0.002965,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
1512173,-0.805935,-0.216782,0.503136,0.115628,-0.419975,-0.662668,-0.413357,1.486282,0.000841,-0.003509,...,-0.002965,0.002719,-0.127087,-0.097503,-0.145898,-0.104474,2.121063,-0.121018,2.043364,2.159477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599225,1.184671,0.424130,0.602589,0.114303,-0.421518,1.505849,-0.401204,-0.471546,-0.009562,-0.007983,...,-0.006451,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
1040938,-0.018731,3.412795,-1.948333,0.079845,3.227068,-0.662668,-0.355628,-0.472445,-0.011049,-0.009102,...,-0.008194,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
1275183,-0.985093,0.428971,-0.007020,-0.471480,-0.399227,-0.662668,-0.368457,-0.453295,0.000841,0.000966,...,-0.001222,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
767898,-1.081674,0.482219,0.386700,-0.470154,-0.399227,-0.662668,2.240495,1.354135,0.005300,-0.001272,...,0.002264,0.002729,0.092219,-0.097503,-0.003563,0.141591,1.958160,-0.121018,1.885100,1.995016


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#  Convolutional Neural Networks

In [15]:
X_train.shape , y_train.shape

((1240540, 83), (1240540, 5))

In [18]:

model_cnn = Sequential()
model_cnn.add(Conv1D(64, 3, padding="same",activation="relu",input_shape=(83, 1)))
model_cnn.add(Conv1D(64, 3, padding="same", activation="relu"))
model_cnn.add(MaxPool1D(pool_size=(2)))
model_cnn.add(Conv1D(128, 3, padding="same", activation="relu"))
model_cnn.add(Conv1D(128, 3, padding="same", activation="relu"))
model_cnn.add(MaxPool1D(pool_size=(2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation="relu"))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(5, activation="sigmoid"))

In [19]:
model_cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001,patience=5) ## early stoppoing

#FIT AND SAVE MODEL FOR FUTURE
model_cnn.fit(X_train, y_train, epochs=1000,validation_data=(X_test, y_test),callbacks=[es])
model_cnn.save("/content/drive/MyDrive/unknown_labelled_cnn.hdf5")

Epoch 1/1000
38767/38767 [==============================] - ETA: 0s - loss: 0.0040 - accuracy: 0.9977

KeyboardInterrupt: ignored

# Accuracies Measure && Model Power

In [ ]:
y_pred_cnn = model_cnn.predict_classes(X_test)
y_probs_cnn=model_cnn.predict_proba(X_test)
np.savetxt('cnn_predictions.txt', np.transpose(np.concatenate((y_test.reshape((y_test.size, 1)),y_pred_cnn), axis=1)), fmt='%01d')
np.savetxt('cnn_prob_predictions.txt', np.around(np.transpose(y_probs_cnn),decimals=5), fmt='%.5f')


In [ ]:
accuracy = accuracy_score(y_test, y_pred_cnn)
print("accuracy:",accuracy)
f1score=f1_score(y_test, y_pred_cnn)
print("f1-acore:",f1score)
cm=confusion_matrix(y_test, y_pred_cnn)
print("confusion matrix:\n",cm)
pr=precision_score(y_test,y_pred_cnn)
print("Precision:",pr)
rs=recall_score(y_test,y_pred_cnn)
print("Recall_score:",rs)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_pred_cnn, labels=model_cnn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model_cnn.classes_)
plt.title("CNN-Confusion Matrix")
plt.show()

## ROC Curves

In [ ]:
from sklearn.metrics import roc_curve, auc,roc_auc_score
plt.figure()
fpr, tpr, thresholds =  roc_curve(y_test, y_probs_cnn)
auc =  roc_auc_score(y_test,y_pred_cnn)
plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % ('CNN', auc))
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('ROC -- Deep Learning Algorithms')
plt.legend(loc="lower right")
plt.rcParams['figure.figsize']=(16,8)
plt.show()   # Display
